In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

/home/maraza/.cache/pypoetry/virtualenvs/graphrag-template-KN6o0p_r-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from dotenv import load_dotenv
load_dotenv()

True

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [3]:
INPUT_DIR = "../inputs/operation dulce"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [4]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 434


[2024-07-08T01:58:45Z WARN  lance::dataset] No existing dataset at /mnt/d/python_projects/graphrag-template/notebooks/../inputs/operation dulce/lancedb/entity_description_embeddings.lance, it will be created


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,entity_type,top_level_node_id,x,y
0,0,ALEX MERCER,PERSON,Alex Mercer is a character with a military bac...,"00fafabae48948779fee2afe600f5143,1e433d6b30887...",1,9,0,b45241d70f0e43fca764df95b2b81f77,9,None,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,TAYLOR CRUZ,PERSON,Taylor Cruz is a character who plays a pivotal...,"00fafabae48948779fee2afe600f5143,1e433d6b30887...",1,12,1,4119fd06010c494caa07f439b333f4c5,12,None,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,JORDAN HAYES,PERSON,Dr. Jordan Hayes is a key character in a narra...,"00fafabae48948779fee2afe600f5143,2cf7a230c367a...",1,9,2,d3835bf3dda84ead99deadbeac5d0d7d,9,None,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,SAM RIVERA,PERSON,Sam Rivera is a character renowned for their t...,"00fafabae48948779fee2afe600f5143,1e433d6b30887...",1,15,3,077d2820ae1845bcbb1803379a3d1eae,15,None,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,PARANORMAL MILITARY SQUAD,ORGANIZATION,The Paranormal Military Squad is an elite clan...,"00fafabae48948779fee2afe600f5143,1e433d6b30887...",1,14,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,14,None,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


#### Read relationships

In [5]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 276


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,ALEX MERCER,TAYLOR CRUZ,7.0,Alex Mercer and Taylor Cruz are integral membe...,"[00fafabae48948779fee2afe600f5143, 1e433d6b308...",b35c3d1a7daa4924b6bdb58bc69c354d,0,9,12,21
1,ALEX MERCER,TAYLOR CRUZ,7.0,Alex Mercer and Taylor Cruz are integral membe...,"[00fafabae48948779fee2afe600f5143, 1e433d6b308...",b35c3d1a7daa4924b6bdb58bc69c354d,0,9,12,21
2,ALEX MERCER,TAYLOR CRUZ,7.0,Alex Mercer and Taylor Cruz are integral membe...,"[00fafabae48948779fee2afe600f5143, 1e433d6b308...",b35c3d1a7daa4924b6bdb58bc69c354d,0,9,12,21
3,ALEX MERCER,TAYLOR CRUZ,7.0,Alex Mercer and Taylor Cruz are integral membe...,"[00fafabae48948779fee2afe600f5143, 1e433d6b308...",b35c3d1a7daa4924b6bdb58bc69c354d,0,9,12,21
4,ALEX MERCER,JORDAN HAYES,6.0,Alex Mercer and Jordan Hayes are colleagues wh...,"[00fafabae48948779fee2afe600f5143, 2cf7a230c36...",a97e2ecd870944cfbe71c79bc0fcc752,1,9,9,18


In [6]:
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

claims = read_indexer_covariates(covariate_df)

print(f"Claim records: {len(claims)}")
covariates = {"claims": claims}

Claim records: 89


#### Read community reports

In [7]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 6


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,4,# Dulce Base and the Paranormal Military Squad...,1,8.5,Dulce Base and the Paranormal Military Squad: ...,The impact severity rating is high due to the ...,"The community is centered around Dulce Base, a...",[{'explanation': 'Dulce Base is the primary lo...,"{\n ""title"": ""Dulce Base and the Paranormal...",6f8ba6b6-506e-46c1-83ce-982d59622554
1,5,# Sam Rivera and the Paranormal Military Squad...,1,7.5,Sam Rivera and the Paranormal Military Squad a...,The impact severity rating is high due to the ...,"The community is centered around Sam Rivera, a...",[{'explanation': 'Sam Rivera is recognized for...,"{\n ""title"": ""Sam Rivera and the Paranormal...",418f4536-d673-4212-8a7c-ca1aac547d0f
2,0,# Dulce Base and the Paranormal Military Squad...,0,8.5,Dulce Base and the Paranormal Military Squad Team,The impact severity rating is high due to the ...,Dulce Base serves as the operational hub for t...,[{'explanation': 'The Paranormal Military Squa...,"{\n ""title"": ""Dulce Base and the Paranormal...",251df57e-fd49-49a7-b262-ccaff95d7a51
3,1,# Dulce Base and the Paranormal Military Squad...,0,8.5,Dulce Base and the Paranormal Military Squad: ...,The impact severity rating is high due to the ...,"The community is centered around Dulce Base, a...",[{'explanation': 'Sam Rivera is recognized for...,"{\n ""title"": ""Dulce Base and the Paranormal...",6e536385-8056-4a82-8670-c0ccaf007fb4
4,2,# Dulce Base: Extraterrestrial Research and Co...,0,8.5,Dulce Base: Extraterrestrial Research and Comm...,The impact severity rating is high due to the ...,Dulce Base is a highly classified facility in ...,[{'explanation': 'Dulce Base is the epicenter ...,"{\n ""title"": ""Dulce Base: Extraterrestrial ...",660c8fba-b65f-4fcd-9c98-203600cf1981


#### Read text units

In [8]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 12


,id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,2cf7a230c367a2dfaf0fc3c903eb8948,# Operation: Dulce\n\n## Chapter 1\n\nThe thru...,2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b35c3d1a7daa4924b6bdb58bc69c354d, a97e2ecd870...","[ad5a2020-cdec-4982-acdf-dbe5ee530066, 9d8a0fe..."
1,6d1255303acb7c9dc951cb0f5fc3042c,be the same.\n\n\*\n\nThe sense of foreboding...,2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b35c3d1a7daa4924b6bdb58bc69c354d, a97e2ecd870...","[5d1c9126-c48d-4755-9f9c-f739c823f95f, ec64a42..."
2,e841f178310356740b2ee9101d12c97f,". ""Your take on these signal inconsistencies?""...",2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b35c3d1a7daa4924b6bdb58bc69c354d, a97e2ecd870...","[0b22a34b-32e9-46a4-a0e8-d3d5466eba15, 7e14972..."
3,f36d96862b9366d7240b5c7ceb04f12b,", absorbed in the bewilderment of contact, whi...",2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b35c3d1a7daa4924b6bdb58bc69c354d, 09f18f81442...","[9cd6d645-ab97-4b39-b02e-647cea9b5545, 50dc124..."
4,f7d43808d2fb452cd953bf50c6de6bd4,"were at once coherent and enigmatic: ""*Voyage...",2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[e02be3e37ca0454883a4c1fd859c24bb, 1dbc51475cb...",[87cf5900-6211-4e04-9115-50f3617c88b4]


In [12]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

### Create local search context builder

In [13]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [14]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [15]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [16]:
result = await search_engine.asearch("Tell me about Agent Mercer")
print(result.response)

Agent Alex Mercer is a prominent figure within the Paranormal Military Squad, known for his leadership and diplomatic role in the squad's mission to establish contact with alien intelligence. Mercer's approach is characterized by a diplomatic demeanor towards potential extraterrestrial contact and a focus on strategic planning [Data: Entities (156); Claims (37, 41, 45, 50, +more)]. His leadership style involves guiding the team through the complexities of interstellar communication efforts, emphasizing the importance of collaboration and decision-making processes [Data: Relationships (0, 5, 8, 4)]. Mercer's seasoned experience and steady resolve make him a central figure in the squad's operations, especially in the critical moments of potential first contact with extraterrestrial beings [Data: Claims (48, 65, 66, 72)]. 

Throughout the data provided, Mercer is depicted as a pivotal member of the team, working closely with other key figures like Taylor Cruz and Dr. Jordan Hayes to navig

In [17]:
question = "Tell me about Dr. Jordan Hayes"
result = await search_engine.asearch(question)
print(result.response)

Dr. Jordan Hayes is a central figure in the Paranormal Military Squad, playing a critical role in decrypting and interpreting alien signals [Data: Entities (2)]. Known for their analytical and skeptical nature, Dr. Hayes contributes to the methodical approach taken by the squad in their communication efforts [Data: Claims (36, 39, 42, +more)]. Their expertise in deciphering alien messages is essential for the squad's mission to establish contact with extraterrestrial intelligence [Data: Relationships (1, 24, 22, 40)].

Dr. Hayes is depicted as a meticulous investigator, clutching the edge of the briefing room table as they analyze constellations and cosmic phenomena, delving into the cosmic aspects of paranormal phenomena [Data: Sources (15, 16); Relationships (23); Claims (2, 7, 34, 46, 64, +more)]. Their role in the squad involves leading efforts in decrypting and interpreting alien signals, showcasing an analytical mindset crucial for the investigation at Dulce Base [Data: Entities 

#### Inspecting the context data used to generate the response

In [18]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,137,WASHINGTON,"The capital of the United States, possibly the...",1,True
1,201,DECRYPTION,Decryption is the process of converting encode...,0,True
2,82,AGENT MICHAEL BROWN,Agent Michael Brown is a high-risk personnel s...,0,True
3,208,YOUTHFUL MEMBER,"A younger individual who is part of a team, br...",0,True
4,128,SILENCE,"A characteristic of the Dulce military base, c...",0,True


In [19]:
result.context_data["relationships"].head()

,id,source,target,description,weight,rank,links,in_context
0,38,SAM RIVERA,PARANORMAL MILITARY SQUAD TEAM,Sam Rivera is a member of the Paranormal Milit...,1.0,22,1,True
1,35,SAM RIVERA,DULCE BASE,"Sam Rivera is part of the Dulce Base team, con...",1.0,32,3,True
2,63,DULCE BASE,PARANORMAL MILITARY SQUAD TEAM,Dulce Base is the location where the Paranorma...,1.0,24,3,True
3,54,EXTRATERRESTRIAL COMMUNICATION,DULCE BASE,Dulce Base serves as a central hub for efforts...,3.0,18,3,True
4,31,SAM RIVERA,PARANORMAL MILITARY SQUAD,Sam Rivera is the youngest member of the Paran...,7.0,29,2,True


In [20]:
result.context_data["reports"].head()

,id,title,content
0,4,Dulce Base and the Paranormal Military Squad: ...,# Dulce Base and the Paranormal Military Squad...


In [21]:
result.context_data["sources"].head()

,id,text
0,3,", absorbed in the bewilderment of contact, whi..."
1,8,ences the full spectrum of human curiosity and...


In [22]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

   id                          entity object_id     status start_date  \
0  46  PARANORMAL MILITARY SQUAD TEAM      NONE       TRUE       NONE   
1  58  PARANORMAL MILITARY SQUAD TEAM      NONE  SUSPECTED       NONE   
2  59  PARANORMAL MILITARY SQUAD TEAM      NONE  SUSPECTED       NONE   
3  83  PARANORMAL MILITARY SQUAD TEAM      NONE  SUSPECTED       NONE   
4  88  PARANORMAL MILITARY SQUAD TEAM      NONE       TRUE       NONE   

  end_date                                        description  in_context  
0     NONE  The team's mission evolved from observation to...        True  
1     NONE  The Paranormal Military Squad team navigates t...        True  
2     NONE  The Paranormal Military Squad team is preparin...        True  
3     NONE  The Paranormal Military Squad team is engaged ...        True  
4     NONE  The Paranormal Military Squad team stood as hu...        True  


### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [23]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [24]:
question_history = [
    "Tell me about Agent Mercer",
    "What happens in Dulce military base?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)

['- What are the key activities conducted by the Paranormal Military Squad at Dulce Base?', '- How do team members like Alex Mercer, Sam Rivera, Taylor Cruz, and Jordan Hayes contribute to operations at Dulce Base?', '- What significance does New Mexico hold in relation to the activities at Dulce Base?', '- How does the secrecy and atmosphere of Dulce Base impact the team members and their mission?', "- What role does the Dulce Military Base play in supporting the Paranormal Military Squad's operations?"]
